In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [2]:
df=pd.read_excel('Course Project - Data for Classification - Electric Vehicles.xls')
#df.head()

In [3]:
#print(df.dtypes)

In [4]:
#print(df['Q16'].unique()) 


In [5]:
df['Q16'].value_counts() 

Q16
A    2883
B    1526
C     892
D     807
Name: count, dtype: int64

In [6]:
# Assuming 'df' is your DataFrame
import numpy as np

# Replace non-numeric values with NaN
df.replace('?', np.nan, inplace=True)
#df.head()

In [7]:
import pandas as pd

# Assuming 'df' is your DataFrame
exclude_column_name = 'Q16'  # Column to exclude from imputation

# Save the excluded column
column_to_exclude = df[exclude_column_name]

# Drop the column you want to exclude for imputation
columns_to_impute = df.drop(columns=[exclude_column_name])

# Impute missing values with median
imputed_data = columns_to_impute.fillna(columns_to_impute.median())

# Create a DataFrame with imputed values
imputed_df = pd.concat([imputed_data, column_to_exclude], axis=1)

# Check the shape of the imputed data
print(f"Imputed data shape: {imputed_df.shape}")


Imputed data shape: (6108, 75)


In [8]:
for column, dtype in imputed_df.dtypes.items():
    print(f"{column}: {dtype}")

Q11_1: int64
Q11_2: int64
Q11_3: int64
Q11_4: int64
Q11_5: int64
Q11_6: int64
Q11_7: int64
Q11_8: int64
Q11_9: int64
Q11_10: int64
Q11_11: int64
Q11_12: int64
Q11_13: int64
Q11_Dont_Know: int64
Q12_1: int64
Q12_2: int64
Q12_3: int64
Q12_4: int64
Q12_5: int64
Q12_6: int64
Q12_7: int64
Q12_8: int64
Q12_9: int64
Q12_10: int64
Q12_11: int64
Q12_12: int64
Q12_13: int64
Q12_14: int64
Q12_Dont_Know: int64
Q13_1: int64
Q13_2: int64
Q13_3: int64
Q13_4: int64
Q13_5: int64
Q13_6: int64
Q13_7: int64
Q13_8: int64
Q13_9: int64
Q13_10: int64
Q13_11: int64
Q13_12: int64
Q13_13: int64
Q13_14: int64
Q13_15: int64
Q13_16: int64
Q13_Dont_Know: int64
Q14: float64
Q15: float64
Q17: float64
Q18_1: float64
Q18_2: float64
Q18_3: float64
Q18_4: float64
Q18_5: float64
Q18_6: float64
Q18_7: float64
Q18_8: float64
Q18_9: float64
Q18_10: float64
Q18_11: float64
Q18_12: float64
Q18_13: float64
Q18_14: float64
Q18_15: float64
Q18_16: float64
Q18_17: float64
Q18_18: float64
Q18_19: float64
Q18_20: float64
Q18_21: floa

In [9]:
print(imputed_df.isnull().sum())

Q11_1     0
Q11_2     0
Q11_3     0
Q11_4     0
Q11_5     0
         ..
Q18_22    0
Q18_23    0
Q20       0
Q21       0
Q16       0
Length: 75, dtype: int64


In [10]:
# from scipy import stats

# # Assuming 'imputed_df' is your DataFrame after imputation
# numeric_columns = imputed_df.select_dtypes(include=['float64', 'int64']).columns
# numeric_df = imputed_df[numeric_columns]

# # Calculate Z-scores for numeric data
# z_scores = stats.zscore(numeric_df)
# threshold = 3
# outlier_indices = (abs(z_scores) > threshold).any(axis=1)
# filtered_df = imputed_df[~outlier_indices]
# imputed_df = filtered_df
# print(f"Shape after removing outliers: {filtered_df.shape}")

In [11]:
import pandas as pd
from sklearn.feature_selection import mutual_info_classif

import pandas as pd
from sklearn.feature_selection import mutual_info_classif
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming df is your DataFrame
target_column = 'Q16'
threshold = 0.2  # Set your desired correlation threshold

# Copy the DataFrame to avoid modifying the original
imputed_df_copy = imputed_df.copy()

# Use mutual_info_classif for feature selection with categorical target
X = imputed_df_copy.drop(columns=[target_column])
y = imputed_df_copy[target_column]

# Calculate mutual information between features and target
mutual_info_values = mutual_info_classif(X, y)

# Create a Series with feature names and their mutual information scores
feature_mutual_info = pd.Series(mutual_info_values, index=X.columns)

# Select features based on the mutual information threshold
selected_features = feature_mutual_info[feature_mutual_info > threshold].index.tolist()

# Rank features by mutual information scores
ranked_features = feature_mutual_info.sort_values(ascending=False)

# Display the ranked features
print("Ranked Features:")
print(ranked_features)

Ranked Features:
Q17       0.155675
Q14       0.139925
Q18_14    0.125440
Q12_14    0.124956
Q21       0.119250
            ...   
Q12_9     0.000000
Q11_9     0.000000
Q11_7     0.000000
Q11_4     0.000000
Q11_1     0.000000
Length: 74, dtype: float64


In [12]:
non_zero_features = ranked_features[ranked_features !=0]

# Display non-zero features and their mutual information scores
print("Non-zero Mutual Information Scores for Features:")
print(non_zero_features)

Non-zero Mutual Information Scores for Features:
Q17       0.155675
Q14       0.139925
Q18_14    0.125440
Q12_14    0.124956
Q21       0.119250
            ...   
Q13_9     0.002936
Q13_12    0.002829
Q11_8     0.002271
Q12_6     0.001808
Q11_3     0.001217
Length: 65, dtype: float64


In [13]:
non_zero_columns = non_zero_features.index.tolist()

# Create a new DataFrame with only the non-zero columns
new_df = imputed_df[non_zero_columns].copy()

# Verify the new DataFrame
print(new_df.head())  # Check the first few rows

   Q17  Q14  Q18_14  Q12_14  Q21  Q13_16  Q18_1  Q18_19  Q18_8  Q18_3  ...  \
0  3.0  1.0     5.0       0    3       0    5.0     5.0    5.0    5.0  ...   
1  3.0  3.0     4.0       0    3       0    4.0     4.0    3.0    2.0  ...   
2  1.0  1.0     4.0       0    1       0    5.0     3.0    4.0    4.0  ...   
3  2.0  2.0     3.0       1    3       1    4.0     3.0    3.0    3.0  ...   
4  2.0  2.0     3.0       0    3       0    4.0     1.0    3.0    3.0  ...   

   Q13_10  Q13_8  Q11_13  Q11_12  Q11_6  Q13_9  Q13_12  Q11_8  Q12_6  Q11_3  
0       0      0       0       0      0      1       0      1      0      0  
1       0      0       0       0      0      1       0      0      0      0  
2       0      0       0       0      0      1       0      1      0      0  
3       0      0       0       0      0      0       0      0      0      0  
4       0      0       0       0      0      0       0      1      0      0  

[5 rows x 65 columns]


In [14]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from collections import Counter

X = new_df  # Feature matrix
y = imputed_df['Q16']  # Target variable
print("Total Count of Data:", X.shape[0])
# Split the data into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the shapes of the resulting sets
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

sm = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = sm.fit_resample(X_train, y_train)

# Check the class distribution after balancing
print("Before SMOTE:", Counter(y_train))
print("Total Count of Data:", X_train_balanced.shape[0])
print("After SMOTE:", Counter(y_train_balanced))

Total Count of Data: 6108
X_train shape: (4886, 65), y_train shape: (4886,)
X_test shape: (1222, 65), y_test shape: (1222,)
Before SMOTE: Counter({'A': 2328, 'B': 1212, 'C': 722, 'D': 624})
Total Count of Data: 9312
After SMOTE: Counter({'D': 2328, 'A': 2328, 'B': 2328, 'C': 2328})


In [15]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import SMOTE

# Assuming 'X' is your features and 'y' is your target variable
# 'X_train_balanced' and 'y_train_balanced' are the balanced training data

# Initialize models
decision_tree = DecisionTreeClassifier(random_state=42)
random_forest = RandomForestClassifier(random_state=42)
gradient_boosting = GradientBoostingClassifier(random_state=42)
gaussian_nb = GaussianNB()

classifiers = {
    'Decision Tree': decision_tree,
    'Random Forest': random_forest,
    'Gradient Boosting': gradient_boosting,
    'Gaussian Naive Bayes': gaussian_nb
}

# Perform 5-fold cross-validation for each classifier using balanced data
for clf_name, clf in classifiers.items():
    # Calculate cross-validated accuracy using balanced data
    scores = cross_val_score(clf, X_train_balanced, y_train_balanced, cv=5)
    avg_accuracy = scores.mean()
    print(f"{clf_name} Accuracy: {avg_accuracy:.4f}")


Decision Tree Accuracy: 0.6657
Random Forest Accuracy: 0.7919
Gradient Boosting Accuracy: 0.7062
Gaussian Naive Bayes Accuracy: 0.5779


In [16]:
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

# Assuming 'X' is your features and 'y' is your target variable
# 'X_train_balanced' and 'y_train_balanced' are the balanced training data

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize models
knn = KNeighborsClassifier()
svm = SVC()
mlp = MLPClassifier(max_iter=1000, solver='adam', learning_rate_init=0.001, tol=1e-4)
logistic = LogisticRegression(max_iter=1000)

classifiers = {
    'KNN': knn,
    'SVM': svm,
    'Neural Network': mlp,
    'Logistic Regression': logistic
}

# Perform 5-fold cross-validation for each classifier using balanced data
for clf_name, clf in classifiers.items():
    # Calculate cross-validated accuracy using balanced data
    scores = cross_val_score(clf, X_train_balanced, y_train_balanced, cv=5)
    avg_accuracy = scores.mean()
    print(f"{clf_name} Accuracy: {avg_accuracy:.4f}")


KNN Accuracy: 0.7392
SVM Accuracy: 0.6670
Neural Network Accuracy: 0.6929
Logistic Regression Accuracy: 0.6478


In [17]:
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

# Assuming 'X' is your features and 'y' is your target variable
# 'X_train_balanced' and 'y_train_balanced' are the balanced training data

# Scaling the data using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize models
knn = KNeighborsClassifier()
svm = SVC()
mlp = MLPClassifier(max_iter=1000, solver='adam', learning_rate_init=0.001, tol=1e-4)
logistic = LogisticRegression(max_iter=1000)

classifiers = {
    'KNN': knn,
    'SVM': svm,
    'Neural Network': mlp,
    'Logistic Regression': logistic
}

# Perform 5-fold cross-validation for each classifier using balanced data
for clf_name, clf in classifiers.items():
    # Calculate cross-validated accuracy using balanced data
    scores = cross_val_score(clf, X_train_balanced, y_train_balanced, cv=5)
    avg_accuracy = scores.mean()
    print(f"{clf_name} Accuracy: {avg_accuracy:.4f}")


KNN Accuracy: 0.7392
SVM Accuracy: 0.6670
Neural Network Accuracy: 0.6995
Logistic Regression Accuracy: 0.6478
